#A guide to the repository

This notebook tries to clarify the purpose of this repository as well as its inner workings. It provides examples to justify the expressions in utils_new.py and illustrate how the functions work. It also explains how to run tests and launch heavier jobs on an HPC server.

In order to use the notebook interactively make sure to have installed the packages defined in requirements.txt.

In [ ]:
import src/utils_new.py

The purpose of this code is to to generate space-time points in the context of Causal Fermion systems, compute the Energy (or action) associated with a given distribution of space-time points to then find the optimal configuration which minimises this action.

#Physical background: The Causal Action Principle
Causal Fermion Systems are a Quantum Gravity theory spearheaded by Felix Finster at Universität Regensburg.

For both introductory and in depth material, see [this website](https://causal-fermion-system.com/);


In Causal Fermion Systems theory, a spacetime point $x$ lives within an $f$-dimensional Hilbert space $\mathcal{H}$ and has specific properties defining the subset $\mathcal{F}$ of space-time points. $x\in\mathcal{F}$ means:

  * $x$ is a hermitian operator
  * $x$ has  at most $n$ positive eigenvalues and $n$ negative eigenvalues yielding  a maximum total of $2n$ non zero eigenvalues. $n$ is called the **spin dimension**
  * **trace constraint**: Trace($x$) = some constant here equal to 1

The Lagrangian for two spacetime point operators $x$ and $y$ is:
\begin{align}
\mathcal{L}(x,y)=\frac1{4n}\sum_{i,j=1}^{2n}\left(|\lambda_i|-|\lambda_j|\right)^2 = \sum_{i=1}^{2n}|\lambda_i|^2-\frac1{2n} \left(\sum_i|\lambda_i|\right)^2.
\end{align}
where $\lambda_i$ are the eigenvalues of the product operator $xy$.

When considering $m$ space-time points, one combines the Lagragians of all couples of spacetime points using a weighted sum, defining the action of the system:

\begin{equation}
    S =\int{\rho}{}{}(x)\int\rho{}{}y \mathcal{L}(x,y) = \sum_{i,j=1}^m c_i c_j\mathcal{L}(x_i,x_j)=\vec c^\intercal (L_{ij}) \vec c.
\end{equation}

with $c$
 the weight vector which emphasizes some space-time points thus defining a measure on $\mathcal{F}$. By the **volume constraint**, the total mass on $\mathcal{F}$ is fixed to 1.

The **Causal Action principle** states that the measure on $\mathcal{F}$ will be such that the action is minimised.

The general purpose of this project is to generate large amounts of space-time points and find the optimal configuration (defined by the coordinates of these points and the value of the weights) to visualise solutions of the Causal Action Principle and gain intuition about the measures which satisfy this principle.

#Specific purpose of this project

More specifically, the purpose of this project is to suggest ways of improving the computation of the action and its minimisation, computing only what is necessary using an efficient parametrisation, taking advantage of the low spin dimension $n$ compared to $f$.

#Structure of the project
*  Generating a spacetime  point
  - Rewriting the Lagrangian to see necessary parameters
  - Parametrising a Unitary Matrix
  - Generating the parameters
  - Building the eigenvectors
* Computing the action
* Implementing a boundary constraint
* Minimising the action
  - Two different solvers: `scipy` and `optimistix`
  - Reshaping the parameters for the solvers
* Running tests
 - Set up
 - Understanding the run (run.py, run_new.py, run_new_optimistix.py) scripts
 - A few test runs
* Running the code on an HPC server
 - Overview
 - Batch job launchers
 - Generating and submitting batch scripts



#Generating a spacetime point
We want to find an efficient way to parametrise a spacetime point $x$ or at least, what is necessary to compute the Lagrangian between two-spacetime points.  Since a spacetime point is a hermitian matrix, it can be written as $ x= UDU^{\dagger}$. We want to  parametrise $D$ and the unitary matrix $U$ such that we only parametrise what is necessary to compute the action. In order to know what we need, let us rewrite the Lagrangian.

## Rewriting the Lagrangian

**Notation**: We write , $x = UDU^{\dagger}$, $y = VEV^{\dagger}$, with:


\begin{align}
V = \begin{bmatrix}
\begin{pmatrix}
\\
\\
v_1\\
\\
\\
\end{pmatrix}
&
\begin{pmatrix}
\\
\\
v_2\\
\\
\\
\end{pmatrix}
 & \cdots
 &
 \begin{pmatrix}
\\
\\
v_f\\
\\
\\
\end{pmatrix}
\end{bmatrix}
\quad
U = \begin{bmatrix}
\begin{pmatrix}
\\
\\
u_1\\
\\
\\
\end{pmatrix}
&
\begin{pmatrix}
\\
\\
u_2\\
\\
\\
\end{pmatrix}
 & \cdots
 &
 \begin{pmatrix}
\\
\\
u_f\\
\\
\\
\end{pmatrix}
\end{bmatrix}
\end{align}

and
\begin{align}
D = \begin{pmatrix} \lambda_1 & & & & \\ & \ddots &&& \\ &&\lambda_{2n} && \\ &&& 0 & \\ &&&& \ddots
    \end{pmatrix}
\quad
E = \begin{pmatrix} \nu_1 & & & & \\ & \ddots &&& \\ &&\nu_{2n} && \\ &&& 0 & \\ &&&& \ddots
    \end{pmatrix}
\end{align}

We want to compute the eigenvalues of $xy$ which we can rewrite as $xy = UDU^{\dagger}VEV^{\dagger}$ which has the same eigenvalues as $ M = DU^{\dagger}VEV^{\dagger}U$ which explicitly exhibits the dot product between the eigenvectors of $x$ and $y$.

Leveraging the zeros in $D$ and $E$, we can write this in a slightly more compact way as $M = GPG^T$
with $G = [u_1,\cdots,u_{2n}][v_1,\cdots,v_{2n}]^{^\dagger}$ a Gram-like matrix of the dot products between the first $2n$ eigenvectors of $x$ and $y$, and $P_{ij} = \lambda_i \nu_j$ the matrix of products between the non-zero eigenvalues of $x$ and $y$.

From this rewriting we see that for each space-time point, all we need is to generate its first $2n$ eigenvectors and its eigenvalues.

##Parametrising a unitary matrix.

Based on work by [Hubert de Guise et al ](https://arxiv.org/pdf/1708.00735) we show that $U$ can be factorised as:

\begin{align}
    U=\left( R_{f,f-1} R_{f-1,f-2} ... R_{21}\right)...\left( R_{f,f-1}...R_{2n-1,2n}\right).
\end{align}

with

\begin{align}
R_i(\alpha_i,\beta_i) = \begin{pmatrix} \exp({i\alpha_i}) \cos\beta_i & - \sin\beta_i \\ \sin\beta_i & \exp({-i\alpha_i})\cos\beta_i\end{pmatrix}    
\end{align}

$U$ can thus be decomposed as a product of factors which we will call **band-matrices** for reasons not detailed here but more easily seen on a circuit diagram representation of the matrices. We index these band-matrices from $1$ to $2n$ with the first band-matrix corresponding to $\left( R_{f,f-1} R_{f-1,f-2} ... R_{21}\right)$

The first band-matrix can be written as:

\begin{align}
   & \left( R_{f,f-1} R_{f-1,f-2} ... R_{21}\right)=
\\&
\begin{pmatrix}
    \exp{i\alpha_1}\cos\frac{\beta_1}2 & -\sin\frac{\beta_1}2 & 0 & \dots \\
    & \ddots & \\
    \exp{i\alpha_k}\cos\frac{\beta_k}2 \prod_{i=1}^{k-1} \sin\frac{\beta_i}2 & & \exp{i(\alpha_k-\alpha_{k-1})}\cos\frac{\beta_k}2 \cos\frac{\beta_{k-1}}2 & -\sin\beta_k/2 & 0 & \dots\\
    \vdots & & \exp{i(\alpha_k-\alpha_{l-1})}\cos\frac{\beta_k}2\cos\frac{\beta_{l-1}}2\prod_{i=l}^{k-1}\sin\frac{\beta_i}2 & \ddots & \dots \\
    \vdots &\dots & \\
    \prod_{i=1}^{f-1} \sin\frac{\beta_i}2 & \dots & \exp{-i\alpha_{l-1}}\cos\frac{\beta_{l-1}}2\prod_{i=l}^{f-1}\sin\frac{\beta_i}2 & \dots & \exp{-i\alpha_{f-1}}\cos\frac{\beta_{f-1}}2
\end{pmatrix}
\end{align}


Similarly, the other band_matrices have the same shape but are progressively shifted to the right and the bottom such that the $i$th band-matrix's first $i$ rows have 1s on the diagonal and 0s everwhere else.

This is seen in the example below:






In [6]:
m = 1; n = 1; f = 4

In [7]:
# @title
seed = 19
key = random.PRNGKey(seed)
subkeys = random.split(key,4)

band_number_1 = 1
band_number_2 =2

alphas_band_1 = jnp.squeeze(random.uniform(subkeys[0],(m,f-band_number_1), minval = 0, maxval = 4*jnp.pi))
betas_band_1 = jnp.squeeze(random.uniform(subkeys[1],(m,f-band_number_1), minval = 0, maxval = jnp.pi/2))

alphas_band_2 = jnp.squeeze(random.uniform(subkeys[2],(m,f-band_number_2), minval = 0, maxval = jnp.pi/2))
betas_band_2 = jnp.squeeze(random.uniform(subkeys[3],(m,f-band_number_2), minval = 0, maxval = 4*jnp.pi))

band_unitary_1 = make_single_band_unitary(alphas_band_1,betas_band_1, f,f)
band_unitary_2 = make_single_band_unitary(alphas_band_2, betas_band_2, f,f)
print("First Band-matrix:")
print(jnp.round(band_unitary_1*1e1)/1e1, end = "\n\n")
print("Second Band-matrix:")
print(jnp.round(band_unitary_2*1e1)/1e1)



First Band-matrix:
[[ 0.6+0.1j -0.8+0.j   0. +0.j   0. +0.j ]
 [-0.3+0.2j -0.2+0.2j -0.9+0.j   0. +0.j ]
 [-0.4+0.6j -0.2+0.5j  0.5-0.1j  0. +0.j ]
 [ 0. +0.j   0. -0.j  -0. +0.j  -0.6-0.8j]]

Second Band-matrix:
[[ 1. +0.j   0. +0.j   0. +0.j   0. +0.j ]
 [ 0. +0.j  -0. -0.4j -0.9+0.j   0. +0.j ]
 [ 0. +0.j   0.4+0.8j -0.4+0.1j  0.1+0.j ]
 [ 0. +0.j  -0.1+0.j   0. -0.1j  0.4-0.9j]]


The takeway message from this is that we will be able to parametrise our unitary matrix using band-matrices for which we have an analytic expression in terms of simple trigonometric functions based on angles $\alpha$ and $\beta$.

A keen eye will notice these terms appear in three combinations of the form:


1.   $\cos{\frac{\beta}{2}}\exp{i\alpha}$
2.   $\cos{\frac{\beta}{2}}\exp{-i\alpha}$
3.   $\sin{\frac{\beta}{2}}$

which we will call **building blocks**.



## Generating the parameters

The parameters we will be needing are thus the positive and negative eigenvalues defining the spectrum of a space-time point, the $\alpha$s and $\beta$s which define its eigenvectors and the weight associated to each space-time point.

These are generated by the `init_params` function. The only caveat is that  the `pos_spectrum` and `neg_spectrum` outputs are not quite yet eigenalues. These must satisfy a trace constraint which we enforce through the `make_spectra` function.

We then convert the $\alpha$s and $\beta$s to the three building blocks which we defined earlier using the `get_building_blocks` function.


In [56]:
weights, pos_spectrum, neg_spectrum, alphas, betas = init_params(key,n,f,m)
spectra = make_spectra(pos_spectrum, neg_spectrum)
cos_betas_exp_pos_alphas, cos_betas_exp_neg_alphas, sin_betas = get_building_blocks(alphas,betas)

In [60]:
# @title
print("Note that the first index gives index of space-time point among the m = {} space-time points".format(m), end = "\n\n")
print("alphas:")
print(alphas, end = "\n\n")
print("spectra: The n = {} positive eigenvalues followed by the n = {}, negative eigenvalues: ".format(n,n))
print(spectra, end = '\n\n')
print("cos_betas_exp_pos_alphas: all the values of the block of cos_betas_exp_pos_alphas")
print(cos_betas_exp_pos_alphas)

Note that the first index gives index of space-time point among the m = 1 space-time points

alphas:
[[4.88847    1.0773908  0.64114255 7.812752   4.402076  ]]

spectra: The n = 1 positive eigenvalues followed by the n = 1, negative eigenvalues: 
[[ 2. -1.]]

cos_betas_exp_pos_alphas: all the values of the block of cos_betas_exp_pos_alphas
[[ 0.06862392-0.3856927j   0.2967985 +0.55190545j  0.06864359+0.05123017j
   0.01394637+0.33806726j -0.10614584-0.3310097j ]]


## Building the eigenvectors:
We have **building blocks** but we need to arange them in certain positions to actually build **band-matrices** and eventually a unitary matrix.

Within a building block, say the building block of all the $\sin{\frac{\beta_i}{2}}$s, each $\sin{\frac{\beta_i}{2}}$ is itself a sub-building block of a band-matrix.

For example, in $\left( R_{f,f-1} R_{f-1,f-2} ... R_{21}\right)$, we identify that $\sin{\frac{\beta_2}{2}}$ appears at the following positions in red:

\begin{align}
   & \left( R_{f,f-1} R_{f-1,f-2} ... R_{21}\right)=
\\&
\begin{pmatrix}
    \exp{i\alpha_1}\cos\frac{\beta_1}2 & -\color{red}{\sin\frac{\beta_1}2} & 0 & \dots \\
    & \ddots & \\
    \exp{i\alpha_k}\cos\frac{\beta_k}2 \prod_{i=1}^{k-1} \color{red}{\sin\frac{\beta_i}2} & & \exp{i(\alpha_k-\alpha_{k-1})}\cos\frac{\beta_k}2 \cos\frac{\beta_{k-1}}2 & -\sin\beta_k/2 & 0 & \dots\\
    \vdots & & \exp{i(\alpha_k-\alpha_{l-1})}\cos\frac{\beta_k}2\cos\frac{\beta_{l-1}}2\prod_{i=l}^{k-1}\sin\frac{\beta_i}2 & \ddots & \dots \\
    \vdots &\dots & \\
    \prod_{i=1}^{f-1} \color{red}{\sin\frac{\beta_i}2} & \dots & \exp{-i\alpha_{l-1}}\cos\frac{\beta_{l-1}}2\prod_{i=l}^{f-1}\sin\frac{\beta_i}2 & \dots & \exp{-i\alpha_{f-1}}\cos\frac{\beta_{f-1}}2
\end{pmatrix}
\end{align}

We can therefore build a **mask matrix** associated with $\sin{\frac{\beta_1}{2}}$ with 1s where there should be a $\sin{\frac{\beta_1}{2}}$ term and 0s elsewhere.

We build similar masks for all $\sin{\frac{\beta_i}{2}}$ and for the other two building blocks.

It turns out the shape of the mask matrices follows relatively simple patterns as a function of $i$ and the index of the band-matrix.

This is performed by the function `make_masks`:

In [61]:
# @title
mask_cos_exp_pos_1, mask_cos_exp_neg_1,mask_sin_1 = make_masks(f,f,band_number = band_number_1)
print("The masks for band-matrix number {} for the sin building block: ".format(band_number_1))
print("")
for i in range(f-band_number_1):
  print("Mask for sin(beta_{}):".format(i+1))
  print(mask_sin_1[i]*1)
  print("")


The masks for band-matrix number 1 for the sin building block: 

Mask for sin(beta_1):
[[0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]]

Mask for sin(beta_2):
[[0 0 0 0]
 [0 0 1 0]
 [1 1 0 0]
 [1 1 0 0]]

Mask for sin(beta_3):
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 1]
 [1 1 1 0]]



We now need to multiply each mask by its associated value and add together the masks of a given band-matrix to get a full band matrix.

This is done by the function `make_single_band_unitary`



In [62]:
# @title
band_matrix = make_single_band_unitary(alphas_band_1,betas_band_1,f,f)

#We now decompose the first steps of make_single_band_unitary for clarity

#extract the building blocks and masks
building_blocks = get_building_blocks(alphas_band_1, betas_band_1)
masks = make_masks(f,f,band_number_1)

#extract masks and sin(beta_i/2) values for the sin building block
mask = masks[2]
building_block = building_blocks[2]

#Multiply the masks and sin values together
band_matrix_sub_building_blocks = mask*building_block[:,jnp.newaxis,jnp.newaxis]

#Combine the different matrices for each sin(beta_i/2)
  # Add 1s where the terms will be non-zero
ones_tril= jnp.tril(jnp.ones((f,f))) + jnp.eye(f,k = 1)
band_matrix_sub_building_blocks_with_ones = band_matrix_sub_building_blocks + ones_tril - mask

  #multiply the matrices together
band_matrix_building_block = jnp.prod(band_matrix_sub_building_blocks_with_ones, axis =0)

print("The masks for the sin building block:", end = "\n\n")
print(mask*1, end = "\n\n")
print("The sin(beta_i/2) values for the sin building block:", end = "\n\n")
print(building_block, end = "\n\n")
print("Each sin(beta_i/2) value mutiplied by associated mask:", end = "\n\n")
for i in range(f-band_number_1):
  print(jnp.round(band_matrix_sub_building_blocks[i]*1e2)/1e2, end = "\n\n")
print("Add 1s where necessary before multiplying all these matrices:", end = "\n\n")
print(jnp.round(band_matrix_sub_building_blocks_with_ones*1e2)/1e2, end = "\n\n")
print("The combination of all these terms in one matrix:", end = "\n\n")
print(jnp.round(band_matrix_building_block*1e2)/1e2, end = "\n\n")
print("Now do the same for the other two building blocks, combine everything, add - signs where necessary and get band-matrix number {}:".format(band_number_1), end = "\n\n")
print(jnp.round(band_matrix*1e2)/1e2)





  #PAS SUR QUE num_masks soit un nom idéal



The masks for the sin building block:

[[[0 1 0 0]
  [1 0 0 0]
  [1 0 0 0]
  [1 0 0 0]]

 [[0 0 0 0]
  [0 0 1 0]
  [1 1 0 0]
  [1 1 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 1]
  [1 1 1 0]]]

The sin(beta_i/2) values for the sin building block:

[0.9098873  0.94499505 0.28154582]

Each sin(beta_i/2) value mutiplied by associated mask:

[[0.   0.91 0.   0.  ]
 [0.91 0.   0.   0.  ]
 [0.91 0.   0.   0.  ]
 [0.91 0.   0.   0.  ]]

[[0.   0.   0.   0.  ]
 [0.   0.   0.94 0.  ]
 [0.94 0.94 0.   0.  ]
 [0.94 0.94 0.   0.  ]]

[[0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.28]
 [0.28 0.28 0.28 0.  ]]

Add 1s where necessary before multiplying all these matrices:

[[[1.   0.91 0.   0.  ]
  [0.91 1.   1.   0.  ]
  [0.91 1.   1.   1.  ]
  [0.91 1.   1.   1.  ]]

 [[1.   1.   0.   0.  ]
  [1.   1.   0.94 0.  ]
  [0.94 0.94 1.   1.  ]
  [0.94 0.94 1.   1.  ]]

 [[1.   1.   0.   0.  ]
  [1.   1.   1.   0.  ]
  [1.   1.   1.   0.28]
  [0.28 0.28 0.28 1.  ]]]

The combination of all t

We can now multiply all the band-matrices together and get the eigenvectors of a space-time point $x$ as the columns of $U$.

Since we only care about the first $2n$ eigenvectors of $x$, we can simplify the caclulation and only consider the first $2n$ columns of the last band-matrix and multiply it through the other band-matrices.

This is performed by the function `make_single_eigenvectors(alphas: jnp.ndarray ,betas:jnp.ndarray,f:int, n:int)` which is then vectorised to handle $m$ space-time points as `make_eigenvectors`.

In [66]:
eigenvectors = make_single_eigenvectors(alphas[0], betas[0],f, n)
print(" eigenvectors: The columns of the matrix are the 2n = {} eigenvectors:".format(2*n), end = "\n\n")
print(jnp.round(eigenvectors*1e2)/1e2)

 eigenvectors: The columns of the matrix are the 2n = 2 eigenvectors:

[[ 0.07-0.39j -0.01-0.31j]
 [ 0.27+0.51j  0.02+0.18j]
 [ 0.05+0.04j -0.9 -0.02j]
 [ 0.71+0.j   -0.24-0.06j]]


#Computing the Action

Now we can start computing the Lagrangian between two space-time points.
We thus use the formulae given above to compute the Lagrangian between two space-time points $x$ and $y$ in `make_lagrangian_n`.

Since we had a special focus on the case $n=1$ we computed a further simplified expression for the Lagrangian in that case, expressed in the function `make_lagrangian_1`.

We then sum the Lagrangians over all pairs of the $m$ spacetime points, taking the weights into account with the `action` function.

In [179]:
# Generate m=3 space-time points, their spectrum and eigenvectors
params = init_params(key,n,f,m=3, sigma_weights = 0.01,init_spectrum = 1, sigma_spectrum = 0.01)
spectra = make_spectra(pos_spectrum = params[1],neg_spectrum = params[2])
eigenvectors = make_eigenvectors(params[3], params[4],f, n)

lag = make_lagrangian_n(spectra,eigenvectors,i=0,j=1)
action_tot = action(params)

print("The Lagrangian between the first (i=0) and second (j=1) space-time points is ", lag)
print("The total action across the m points is" , action_tot)

The Lagrangian between the first (i=0) and second (j=1) space-time points is:  0.54841936
The total action across the m points is: 1.6683025


#The boundedness constraint

We can add another constraint to the optimisation problem. Until now, volume and trace constraints had been implemented directly in the definition of the parameters, letting us deal with an unconstrained optimisation problem. It will now be difficult to do the same with the boundedness constraint which simply imposes some  upper bound on the weighted sum of the eigenvalues of all pairs of space-time points as follows:

\begin{equation}
\mathcal{T}(\rho) := \iint_{\mathcal{F} \times \mathcal{F}}
\left( \sum_{i=1}^{2n} |\lambda_i^{xy}| \right)^2
d\rho(x) \, d\rho(y) \leq C
\end{equation}

It is computed using the function `boundedness(params: Params)`

It is difficult to implement this as an a priori constraint on the parameters as one needs to take into account the eigenvalues of all couples of space-time points.

We thus implement this highly non-linear constraint using a barrier term
\begin{equation}
\mathcal{L}_{bounded}= \mathcal{L}-\frac{1}{k}\log(C - \tau)
\end{equation}

where we chose $k = -10^3\log({0.1})mn^3$

This loss is computed by the function `action_with_barrier_bnd_constraint`

Note however that for this barrier to be well defined, $\tau$ must be smaller than $C$ from the start. It is hence necessary to initialise the parameters in such a way that they satisfy the boundedness constraint.

This is called **feasibility initialisation**.


For this purpose, before minimising the causal action we drive the parameters in the right region by first solving a different optimisation problem whose minimum satisfies the boundedness constraint.

We thus minimise the loss:
\begin{equation}
\mathcal{L}_{feasibility} = \max(\tau-C,0)
\end{equation}.

This is implemented by the functions `feasibility_cost` which defines this loss and `satisfy_feasibility` which solves the feasibility optimisation problem within the class `Optimistix_BFGS_Solver`.

#Optimising the action

## Two different solvers

We build two alternative solvers, one  using `jax.scipy.minimise.lbfgs` and the other using the library `optimistix`.

As can be read at the [following adress](https://docs.kidger.site/optimistix/faq/), `jax.scipy.minimize` is deprecated and is not at the state of the art.

Although we have seen that on comparable tests (small $f$, large $m$) `optimistix` clearly outperforms `jax.scipy.minimise.lbfgs`, at the time of this project the LBFGS algorithm was not implemented on `optimistix`. For large values of $f$, the intrinsic speedup of LBFGS compared to BFGS thus makes `jax.scipy.minimise.lbfgs` still valuable.

There are thus two optimiser functions in this code: `optimize_scipy ` and `optimize_optimistix`.

1. `optimize.scipy`:
  -  Unlike optimistix which automatically computes the gradient, `scipty.minimise` takes as input both the cost function and its gradient.
  - Our optimiser works in two steps. First it implements lbfgs for a certain number of steps for speed, and then it applies bfgs to find a precise solution.
  - A detailed `Callback` class enables us to collect datapoints about the evolution of the optimisation by  storing the parameter values and the action every `checkpoint_freq`optimisation steps.

2. `optimize_optimistix` is built on top of a class `Optimistix_BFGS_Solver`
  - This class enables us to define a tailor-made solver which can minimise the causal action with or without a boundedness constraint.
  - It defines the method `_optimise` which simply implements `optimistix.BFGS` on a given cost_function.
  - The methods `minimise_unconstrained_action` and `minimise_constrained_action` then  apply `_optimise` with the appropriate loss function (and with the initial `satisfy_feasibility` step in the constrained case).
  - Defining Callback functions on optimistix is less straightforward thus  there is no Callback function here

## Reshaping the inputs and outputs for the optimisers

Optimisers need the parameters to be flattened. We thus build the functions `_flatten_params`, `_reconstruct_params`, `action_flat_params`.

The flattened parameters and `action_flat_params` will be given as an argument to the solvers which will then try to find the optimal parameters.

Optimistix requires the cost function to be written slightly differently as it must have as input both the parameters which are optimised and the constant arguments `args` hence the functions `_feasibility_cost_with_args` or `_action_with_args`

Regarding outputs, when runnning the optimisation, scipy.minimise and optimistix have outputs of a different format. There are hence parts of the code of `optimize_optimistix` and `optimize_scipy` which reshape this output so that both optimisers have a similar output.




In [8]:
# @title
m = 2; n = 1; f = 4
seed = 19
key = random.PRNGKey(seed)

lbfgs_maxiter = 10000
lbfgs_gtol = 1e-7
lbfgs_ftol = 1e-9
lbfgs_maxcor = 70
lbfgs_maxls = 20

bfgs_maxiter = 10000
bfgs_rtol= 1e-7
bfgs_atol= 1e-9
bfgs_gtol = 1e-7

sigma_weights = 0.01
init_spectrum = 1
sigma_spectrum = 0.01

lbfgs_options = {
    "maxiter": lbfgs_maxiter,
    "maxfun": 2 * lbfgs_maxiter,
    "disp": 50,
    "gtol": lbfgs_gtol,
    "ftol": lbfgs_ftol,
    "maxcor": lbfgs_maxcor,
    "maxls": lbfgs_maxls,
  }
bfgs_options = {
"maxiter": bfgs_maxiter,
"disp": True,
"gtol": bfgs_gtol,
}



In [18]:
# @title
key, subkey = random.split(key)
params_0 = init_params(subkey, n, f, m,sigma_weights,init_spectrum,sigma_spectrum)
#optimistix:
final_params_optimistix, bfgs_res_optimistix = optimize_optimistix( params_0, n, f, m, bfgs_maxiter, bfgs_rtol, bfgs_atol)

#scipy

  #We bypass the callback function in this simple example by setting a very large checkpoint frequency.
  #We will thus not need an output directory for the Callback function:

checkpoint_freq = lbfgs_maxiter + bfgs_maxiter
out_dir  = None

final_params_scipy, bfgs_res_scipy = optimize_scipy(
    params_0, n, f, m, lbfgs_options, bfgs_options, out_dir,
    checkpoint_freq)


/usr/local/lib/python3.11/dist-packages/scipy/optimize/_minimize.py:708: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: 0.250000
         Iterations: 0
         Function evaluations: 71
         Gradient evaluations: 60


In [23]:
# @title
print("optimistix:", end  = "\n\n")
print("final_params_optimistix:", final_params_optimistix)
print("bfgs_res_optimistix: ", bfgs_res_optimistix, end = "\n\n")

print("scipy:", end  = "\n\n")
print("final_params_scipy:", final_params_scipy)
print("bfgs_res_scipy: ", bfgs_res_scipy)

optimistix:

final_params_optimistix: (Array([0.00560452, 0.00542667], dtype=float32), Array([[8.335797],
       [7.067811]], dtype=float32), Array([[-7.6338406],
       [-6.397291 ]], dtype=float32), Array([[ 6.070735 ,  7.2938066,  3.4974928, 10.937419 ,  5.341822 ],
       [ 1.5411974, 12.518247 ,  1.6796926,  1.5048906,  8.7280655]],      dtype=float32), Array([[0.7808373 , 0.94757605, 0.35858634, 0.03970427, 1.3226327 ],
       [1.1094694 , 1.5514561 , 0.7609991 , 0.9170569 , 0.489293  ]],      dtype=float32))
bfgs_res_optimistix:  {'action': array([0.25000098], dtype=float32), 'boundedness': array([0.50000197], dtype=float32), 'n_iterations': array([53], dtype=int32), 'n': array(1), 'f': array(4), 'm': array(2)}

scipy:

final_params_scipy: (Array([0.00471749, 0.0043963 ], dtype=float32), Array([[8.048061],
       [8.532693]], dtype=float32), Array([[-7.3448763],
       [-7.862127 ]], dtype=float32), Array([[ 6.090608 ,  7.299733 ,  3.4826152, 10.937173 ,  5.3417554],
       [ 1.

In [22]:
# @title
#scipy
print("scipy:", end = "\n\n")
print("final action:", bfgs_res_scipy["action"])
print("number of lbfgs iterations:", bfgs_res_scipy['lbfgs_iterations'])
print("number of bfgs iterations:",bfgs_res_scipy['bfgs_iterations'])
print("Final weights: ", final_params_scipy[0], end = "\n\n")
#optimistix
print("optimistix:", end = "\n\n")
print("final action:", bfgs_res_optimistix["action"])
print("number of iterations:", bfgs_res_optimistix['n_iterations'])
print("Final weights: ", final_params_optimistix[0])

scipy:

final action: [0.25000018]
number of lbfgs iterations: 25
number of bfgs iterations: 0
Final weights:  [0.00471749 0.0043963 ]

optimistix:

final action: [0.25000098]
number of iterations: [53]
Final weights:  [0.00560452 0.00542667]


#Running test optimisation:


##Set up

Set up a virtual environment  using

 `conda create -n cfs_v2024_env python=3.10`

Then activate that environment using

 `conda activate cfs_v2024_env `

Navigate to the cfs_numerics_v2024 folder then install the requirements using

`pip install -r requirements.txt`

##Understanding the run python scripts

There are three different run python scripts:
- run.py: uses the parametrisation built by Niki Kilbertus in [the original paper](https://arxiv.org/pdf/2201.06382v1)
- run_new.py: implements the new parametrisation using the scipy.minimise optimiser
- run_new_optimistix.py: implements the new parametrisation using optimistix

###Inputs

Since we want to run the optimisation automatically using scripts, we use absl flags to define variables which we can modify in scripts.

For example,  in run.py one can see:
`flags.DEFINE_integer("n", 2, "The spin dimension.")`
defines an integer  variable $n$ with an initial value of 2, labeled as "the spin dimension"

We can modify this variable when we call run.py in the command line with a script such as:

`python ./src/run.py --f=2 --n=1 --m=16 --seed=4`

where `--n = 1` inputs a different value to the flag.

###Outputs

All three run files will store their output in the directory `outdir = ..\result` stored 1 directory up relative to where the run files are run. If that directory does not exist yet it is made using

`if not os.path.exists(out_dir):
    os.makedirs(out_dir)`

There are three kinds of outputs:
* loggings
  - the absl library enables one to keep track of what is being done during the calculations by writing logings while the program is run both in the console and in a separate file.
    - `FLAGS.alsologtostderr = True` ensures the loggings are written in the console
    - `FLAGS.log_dir = out_dir` defines wehre the logs are stored (FLAGS.log_dir is an absl keyword)
    - `logging.get_absl_handler().use_absl_log_file(program_name="run")`stores the loggings as `run.INFO`
      - All the content in `logging.info("content")` will be stored in a file somewhat like `run.INFO`
* checkpoints:
  - the scipy optimiser calls a checkpoint function:
    `final_params, bfgs_res = utils_new.optimize(
    params_0, FLAGS.n, FLAGS.f, FLAGS.m, lbfgs_options, bfgs_options, out_dir,
    FLAGS.checkpoint_freq)`

    Here, `outdir` and `checkpoint_freq` are inputs to `write_checkpoint`which is defined in utils.py or utils_new.py. Every `checkpoint_freq` iterations, it stores:
      - the current raw parameter values :
     `np.savez(result_path, weights=weights, pos_spectrum=pos_spectrum neg_spectrum=neg_spectrum, alphas=alphas, betas=betas)`
      - the current meaningful parameters, reshaping the raw parameters as eigenvectors, eingenvalues...This reshaping is performed by the `collect_results` function in utils_new.py  which outputs a dictonary called results with all this interpretable information
      `results.update(collect_results(params))
       np.savez(result_path, **results)`

      - While the raw parameters are saved as a separated .npz file at each step, the result dictionary is overwritten at each step


* final output:
   - We use the `write_checkpoint` function to store the optimal parameters and add $n$,$f$, $m$ and the results of the optimisation to the `results dictionary`:
  - ```results = defaultdict(list)
      results['n'] = FLAGS.n
      results['f'] = FLAGS.f
      results['m'] = FLAGS.m

      final_params, bfgs_res = utils.optimize(....)
      results.update(bfgs_res)
      utils.write_checkpoint(final_params, 'parameters_last', out_dir, results)
    ```

##A few test runs:
With working directory at cfs_numerics_v2024:

- checking run_new.py works:`python ./src/run_new.py --f=2 --n=1 --m=16 --seed=4`
- checking run_new_optimistix.py works without boundedness constraint:`python ./src/run_new_optimistix.py --f=2 --n=1 --m=16 --seed=4`
- checking run_new_optimistix.py works with boundedness constraint: `python ./src/run_new_optimistix.py --f=2 --n=1 --m=16 --seed=4 --boundedness=2`


#Running the code on an HPC server

When one connects to a server, one typically has acces to two types of nodes:

- A **login node** where you arrive when you connect to the server. It is used to  launch jobs, run small tests, manage files: everything other than heavy computations.
- **compute nodes**: The nodes on which to launch the heavy computations. One does not have direct access to them and must submit a job to SLURM, a job scheduler which whill manage resources between all the submitted jobs efficiently.

##Overview:
This repository contains shell scripts which are run on the login node:

- setup_env.sh
- run_test_job.sh
- run_gabs_test.sh
- run_all_experiments.sh

The first two enable us to ensure our  working repository has all the required installs and works properly.

The last two are called **batch job launchers** and make the link between the login and compute nodes.

They call the  python script cluster_submit.py who  creates **batch scripts** and submits them to SLURM.

SLURM will read through these batch scripts and execute the task defined in them on the compute nodes.

These tasks in our case are the optimisation problems defined by:
- run.py
- run_new.py
- run_new_optimistix.py

with hyperparameters $m$, $f$, $n$... defined in configs.py.

## Getting started

Once connected to the server, navigate to your working directory and in a bash shell run the shell script
setup_env.sh to create a conda environement where the requirements defined in requirements.txt are satisfied.

##Batch job launchers

###Testing
You can now run `run_test_job.sh`.

This is a script that will test whether you server environment works well and run a test run on the login node (not on the compute nodes which are managed by SLURM).

It does not need cluster_submit.py (whose purpose is to  generate bash scripts) and hence run_test_job.sh directly writes a shell script which activates the conda environment and launches run.py with a simple parameter configuration defined directly in the script:

`python ./src/run.py --f=2 --n=1 --m=16 --seed=4`

###Running experiments

If this works well, you can then run any batch job launcher such as run_all_experiments.sh or run_gabs_test.sh

These will run cluster_submit.py based off parameter configurations defined in configs.py:

They contain the following commands:

- In run_gabs_test.sh:

`python "${script_dir}/src/cluster_submit.py" --experiment_name=n1_testing_0712 --config=gabs_test --n_runs 1 --runpath=/home/x_rojon/cfs_numerics_v2024/src/run.py --result_dir=/home/x_rojon/results/`

- In run_all_experiments.sh:

`python src/cluster_submit.py --experiment_name=n1_large_f --config=n1_large_f --n_runs 1`

This shows that cluster_submit.py requires

- an experiment name to store the experiment under an appropriate name
-  a configuration such as `gabs_test` or `n1_large_f` which are both defined in config.py
- the path to the python script that we want to run `runpath`: here the optimisation procedure so runpath can lead to either run.py, run_new.py or run_new_optimistix.py
    - you can see in run_gabs_test.sh that the `runpath` refers to the cloned cfs_numerics_v2024 repository on the user's directory on the server and then points to run.py
- an output directory `result_dir` so that the server knows where to store the results

**Note that run_all_experiments.sh does not specify a `runpath` or a `result_dir`.  It is a legacy code which should be updated  to run correctly**.

##Generating and submitting batch scripts: cluster_submit.py

When it is called by the above batch job launchers,
cluster_submit.py will generate and submit batch scripts to SLURM.

It generates a  SLURM batch script (run_job.cmd) which requests resources  (CPU, GPU, time, memory), activates the conda environment, and runs the optimisation Python script.

(You can chose that  optimisation python script among run.py, run_new.py or run_new_optimistix.py by modifying the `runpath` flag when calling cluster_submit)

cluster_submit.py generates the script through the function `submit_all_jobs` which defines a sequence of lines and then calls

`with open("run_job.cmd", "w") as file:
      file.write("\n".join(lines))`

to create the script under the appropriate name and put together all the lines in the script.

It then submits it to slurm using `os.system("sbatch run_job.cmd")`




